# Scraping Twitter with snscrape

This noteboook is a complimentary notebook to the course project notebook for ST451.

It contains functions that are used for scraping Twitter via `snscrape`.

First, we installed the package `snscrape` from its GitHub page.

pip install git+https://github.com/JustAnotherArchivist/snscrape.git

In [1]:
import numpy as np
import pandas as pd
import math
import datetime
import re

import snscrape.modules.twitter as sntwitter

import random


def queries(hashtag,year,num_months):
    queries = []
    for mon in range(num_months):
        #random_triple = random.sample(range(28), num_days)
        #len_mon = 30 if (mon== 0,2)
        for day in range(28):
            date = np.datetime64(f'{year}-{str(mon+1).zfill(2)}-{str(day+1).zfill(2)}')
            query = f'#{hashtag} since:{date} until:{date+1}'
            queries.append((hashtag,query))
    return(queries)




def scraper_f(query):
    tweets_list = []
    try:
        for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query[1]).get_items()):
            if i >= 100:
                break
            tweets_list.append((query[0],tweet))
    except:
        pass
    
    return(tweets_list)
     

In [3]:
import pickle
import time

print(time.time())
list_of_lists = [queries("SuezCanal",year,12) for year in range(2018,2022)]
queries_list = [element for list_queries in list_of_lists for element in list_queries]
daytweets_lists = list(map(scraper_f,queries_list))


1619279384.801274


In [4]:
pickle.dump( daytweets_lists, open( f'Suez_Canal_2018-2021.p', "wb" ) )
print(time.time())
#time.sleep(15)


1619281163.75984


Time taken to implement the code in the cell above.

In [6]:
t = (1619281163.75984-1619279384.801274)/60
print(t)

29.649309432506563


In [ ]:
# the below code is for creating a csv file with tweets
def en_tweets_info(file_name): 

    daytweets = pickle.load(open(file_name,"rb"))
    daytweets_flat = [element for a_list in daytweets for element in a_list]
    tweetslist = [[tweet_tuple[0], 
                   tweet_tuple[1].date, tweet_tuple[1].id, #tweet information
                   tweet_tuple[1].content, tweet_tuple[1].lang,
                   tweet_tuple[1].user.username, tweet_tuple[1].user.location] 
                  for tweet_tuple in daytweets_flat if tweet_tuple[1].lang =="en"]
    
    return tweetslist

tweetsdf = pd.DataFrame([x[3] for x in en_tweets_info('Suez_Canal_2018-2021.p')], 
                                     columns = ['Content']) # just need the column with contents
tweetslists = tweetsdf.to_csv("tweets.csv")
